<a href="https://colab.research.google.com/github/fourthbrain-students/glg-ack/blob/main/1.1-cgc-GrailQA_With_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GrailQA Dataset Exploration w/ BERT (w/ domain selection and mapping)

Adpated from https://towardsdatascience.com/multi-label-multi-class-text-classification-with-bert-transformer-and-keras-c6355eccb63a

In [34]:
!pip install transformers
from transformers import TFBertModel,  BertConfig, BertTokenizerFast

from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.math import confusion_matrix
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy, Precision, Recall
from tensorflow.keras.utils import to_categorical

import pandas as pd
from sklearn.model_selection import train_test_split

In [35]:
!pip install tensorflow_addons
from tensorflow_addons.metrics import F1Score

In [36]:
!pip install datasets
from datasets import load_dataset
dataset = load_dataset(
   'grail_qa')

Using custom data configuration default
Reusing dataset grail_qa (/root/.cache/huggingface/datasets/grail_qa/default/0.0.0/2be99c6902e842f3ba87dd36fa96a2236206438ebb67c5e5464a36e4049fd3fb)


In [37]:
df_train = pd.DataFrame(dataset['train'])
df_valid = pd.DataFrame(dataset['validation'])
df_test = pd.DataFrame(dataset['test'])

In [38]:
df_train.head()

,qid,question,answer,function,num_node,num_edge,graph_query,sparql_query,domains,level,s_expression
0,2101535001000,oxybutynin chloride 5 extended release film co...,"{'answer_type': ['Entity', 'Entity'], 'answer_...",none,2,1,"{'nodes': {'nid': [0, 1], 'node_type': ['class...",PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,[medicine],,(AND medicine.routed_drug (JOIN medicine.route...
1,2100954014000,the type single-sex school are in which instit...,"{'answer_type': ['Entity', 'Entity', 'Entity',...",none,2,1,"{'nodes': {'nid': [0, 1], 'node_type': ['class...",PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,[education],,(AND education.educational_institution (JOIN e...
2,3206374001000,the leaders of the earliest established religi...,"{'answer_type': ['Entity'], 'answer_argument':...",argmin,3,2,"{'nodes': {'nid': [0, 1, 2], 'node_type': ['cl...",PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,[religion],,(ARGMIN religion.religious_leadership_title (J...
3,2100735000000,"on 07/01/1970, which warship v1.1 was hit?","{'answer_type': ['Entity'], 'answer_argument':...",none,2,1,"{'nodes': {'nid': [0, 1], 'node_type': ['class...",PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,[user.patrick.default_domain],,(AND user.patrick.default_domain.warship_v1_1 ...
4,2102770001000,what is the language regulator of basque?,"{'answer_type': ['Entity'], 'answer_argument':...",none,2,1,"{'nodes': {'nid': [0, 1], 'node_type': ['class...",PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,[language],,(AND language.language_regulator (JOIN languag...


The test partition has empty lists for the 'domains' column 🤔

In [ ]:
df_train['label'] = pd.Categorical([domains[0] for domains in df_train['domains']])
df_valid['label'] = pd.Categorical([domains[0] for domains in df_valid['domains']])

In [43]:
len(df_train[(df_train['label'] == 'medicine')])

2013

In [44]:
domains_to_keep = ['medicine', 'computer', 'spaceflight', 'biology', 'automotive', 'internet', 'engineering']
df_train = df_train[df_train['label'].isin(domains_to_keep)]
df_valid = df_valid[df_valid['label'].isin(domains_to_keep)]

In [45]:
domain_map = {
    'medicine': 'healthcare',
    'computer': 'technology',
    'spaceflight': 'technology',
    'biology': 'healthcare',
    'automotive': 'technology',
    'internet': 'technology',
    'engineering': 'technology'
}
df_train['label'] = df_train['label'].map(domain_map)
df_valid['label'] = df_valid['label'].map(domain_map)

In [47]:
df_train['label'] = pd.Categorical([label for label in df_train['label']])
df_valid['label'] = pd.Categorical([label for label in df_valid['label']])
df_train['numeric_label'] = df_train['label'].cat.codes
df_valid['numeric_label'] = df_valid['label'].cat.codes

In [48]:
print(len(df_train['label'].unique()))
df_train['label'].value_counts()

2


technology    4989
healthcare    3286
Name: label, dtype: int64

### Fetch BERT pre-trained encoder and tokenizer

In [49]:
#######################################
### --------- Setup BERT ---------- ###
# Name of the BERT model to use
model_name = 'bert-base-uncased'
# Max length of tokens
max_length = 100
# Load transformers config and set output_hidden_states to False
config = BertConfig.from_pretrained(model_name)
config.output_hidden_states = False
# Load BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)
# Load the Transformers BERT model
transformer_model = TFBertModel.from_pretrained(model_name, config = config)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [51]:
#######################################
### ------- Build the model ------- ###
# TF Keras documentation: https://www.tensorflow.org/api_docs/python/tf/keras/Model
# Load the MainLayer
bert = transformer_model.layers[0]
# Build your model input
input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
inputs = {'input_ids': input_ids}
# Load the Transformers BERT model as a layer in a Keras model
bert_model = bert(inputs)[1]
dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
pooled_output = dropout(bert_model, training=False)
# Then build your model output
label = Dense(units=len(df_train['numeric_label'].value_counts()),
              kernel_initializer=TruncatedNormal(
                  stddev=config.initializer_range
                  ), 
              name='label')(pooled_output)
outputs = {'label': label}
# And combine it all in a model object
model = Model(inputs=inputs, outputs=outputs, name='BERT_MultiClass')
# Take a look at the model
model.summary()

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Model: "BERT_MultiClass"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 100)]             0         
_________________________________________________________________
bert (TFBertMainLayer)       TFBaseModelOutputWithPool 109482240 
_________________________________________________________________
pooled_output (Dropout)      (None, 768)               0         
_________________________________________________________________
label (Dense)                (None, 2)                 1538      
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


### Model Training

In [53]:
optimizer = Adam(
    learning_rate=5e-05,
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0
    )

loss = {'label': CategoricalCrossentropy(from_logits = True)}
metric = {'label': CategoricalAccuracy('accuracy')}

model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

y_label = to_categorical(df_train['numeric_label'])

x = tokenizer(
    text=df_train['question'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids=False,
    return_attention_mask=False,
    verbose=True)

history = model.fit(
    x={'input_ids': x['input_ids']},
    y={'label': y_label},
    validation_split=0.2,
    batch_size=64,
    epochs=2
)

Epoch 1/2
104/104 [==============================] - 69s 459ms/step - loss: 0.0082 - accuracy: 0.9982 - val_loss: 0.0097 - val_accuracy: 0.9976
Epoch 2/2
104/104 [==============================] - 45s 435ms/step - loss: 0.0021 - accuracy: 0.9995 - val_loss: 0.0048 - val_accuracy: 0.9988


In [54]:
model.save('model.hdf5')

### Evalutation

Using the dev partition since the test partition is unlabeled.

In [55]:
test_y = to_categorical(df_valid['numeric_label'])
test_x = tokenizer(
    text=df_valid['question'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding="max_length",
    pad_to_max_length=True,
    return_tensors='tf',
    return_token_type_ids=False,
    return_attention_mask=False,
    verbose=True)

model_eval = model.evaluate(
    x={'input_ids': test_x['input_ids']},
    y={'label': test_y}
)

23/23 [==============================] - 2s 82ms/step - loss: 0.0017 - accuracy: 0.9986


### Verify Accuracy

In [56]:
preds = model.predict(x={'input_ids': test_x['input_ids']})

In [58]:
from tensorflow.math import argmax
correct = 0
for pred, expected in zip(argmax(preds['label'], axis=1),
                          df_valid['numeric_label']):
  if pred == expected:
    correct += 1
print(f"Accuracy: {correct / len(df_valid)}")

Accuracy: 0.9985974754558204


### Confusion Matrix

In [59]:
from tensorflow import argmax
confusion_matrix(df_valid['numeric_label'],
                 argmax(preds['label'], axis=1),
                 num_classes=len(df_valid['label'].unique()))

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[303,   1],
       [  0, 409]], dtype=int32)>